In [1]:
import numpy as np
import pandas as pd
import logging
import os
from datetime import timedelta
from pathlib import Path

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now
from tinkoff.invest.caching.market_data_cache.cache import MarketDataCache
from tinkoff.invest.caching.market_data_cache.cache_settings import (
    MarketDataCacheSettings,
)

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import sys
sys.path.append("..") 
from Portfolio.portfolio_tools import *
from tp_config import TINK_DATA
import tink_port as tink

Вводим токен, получаем список доступных аккаунтов

In [2]:
BASE = 't.UFRJ8SC9hafVOhFxEUY7yf1wZ1gGhwJp-WCp9o4rnEChHWns0c3jQ21eQwoOW_RurFqeZpss2scJkmMQnomJ9g'
ETF = 't.6nHltT1dYSfrVTIV9zF72fxDlB2sXJbRD6iJNpZXTFAN61rmD7m71xPp9ko12ta1JxA06em4YdN36xicnBmjWg'
MOMENTUM = 't.24WV5_MMG1bQArK1WPp1_DYWD52f-VfGjpR1ci5Pqf0PJ948zWhDstoO_6d4wXIhFTMVsVJSgOzPElUIPEO4Mw'

token = MOMENTUM

In [3]:
accs = tink.get_accounts(token)
print("Количество аккаунтов:", len(accs.accounts))

print(accs.accounts[0].name)
account_id = accs.accounts[0].id

Количество аккаунтов: 1
Momentum 90 Days


#### База идентификаторов Tinkoff

In [4]:
base = tink.get_id_base(token)
port = tink.get_portfolio(token)
df_port = tink.port_to_df(port, base)

dfx = base[base["type"] == "shares"]
dfx = dfx[dfx["cur"] == "rub"]
base_ru = dfx.copy()

In [5]:
df_port['sums'] = df_port.quantity * df_port.price
df_port

,figi,ticker,name,quantity,lot_quantity,price,sums
4,RUB000UTSTOM,0-RUB,None,47946,47946,1.00,47946.0
6,TCS10A0JNAB6,ABIO,Артген,10,1,118.86,1188.6
1,BBG004S68614,AFKS,АФК Система,200,2,24.00,4800.0
0,BBG004S683W7,AFLT,Аэрофлот,80,8,49.27,3941.6
3,BBG012YQ6P43,CIAN,Циан АДР,3,3,859.00,2577.0
9,TCS00A107J11,DELI,Делимобиль,8,8,346.00,2768.0
7,TCS2207L1061,HHRU,HeadHunter Group PLC,1,1,4430.00,4430.0
2,BBG0063FKTD9,LENT,Лента,7,7,962.50,6737.5
8,TCS00A103X66,POSI,Positive Technologies,1,1,2817.60,2817.6
5,BBG00F9XX7H4,RNFT,РуссНефть,135,135,244.30,32980.5


### Portfolio Black-Litterman Model

In [8]:
mcaps = {}
for t in dfp.columns:
   # mcaps[t] = stock.info["marketCap"]
    mcaps[t] = 1
mcaps

{'LKOH': 1,
 'NLMK': 1,
 'AGRO': 1,
 'MAGN': 1,
 'TATN': 1,
 'SIBN': 1,
 'BANEP': 1,
 'SNGSP': 1,
 'POSI': 1,
 'SBER': 1,
 'CHMF': 1,
 'ROSN': 1}

In [9]:
mcaps['SBER'] = 1.5
mcaps['POSI'] = 1.5

In [28]:
imoex_figi = 'BBG333333333'
market_prices = tink.get_candles(token, imoex_figi, CandleInterval.CANDLE_INTERVAL_DAY)
market_prices =  tink.get_open_price(market_prices)

In [21]:
dfp

,LKOH,NLMK,AGRO,MAGN,TATN,SIBN,BANEP,SNGSP,POSI,SBER,CHMF,ROSN
date,,,,,,,,,,,,
2023-10-02,6716.5,210.50,1208.0,52.650,628.0,722.10,1468.0,58.500,2372.2,261.37,1371.0,538.50
2023-10-03,6720.0,209.96,1190.0,52.955,639.0,711.00,1439.5,56.800,2420.0,258.99,1392.0,534.65
2023-10-04,6740.0,208.86,1179.8,51.500,640.0,723.90,1432.5,56.650,2397.0,259.00,1355.6,538.20
2023-10-05,6706.0,212.84,1149.8,52.250,634.2,704.45,1475.0,55.475,2440.0,260.70,1370.0,536.00
2023-10-06,6749.5,210.80,1158.8,51.995,635.0,718.10,1460.0,57.090,2435.0,259.77,1369.4,534.50
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-21,7450.0,212.78,1411.8,54.800,695.7,755.70,2142.0,61.845,2674.0,295.99,1743.2,561.30
2024-03-22,7420.0,217.98,1422.6,55.520,704.1,756.80,2143.0,62.490,2680.0,295.77,1796.6,562.85
2024-03-25,7305.0,214.50,1388.0,54.800,694.5,737.00,2120.0,61.945,2642.2,294.00,1777.8,556.70


In [38]:
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel

S = risk_models.CovarianceShrinkage(dfp).ledoit_wolf()
delta = black_litterman.market_implied_risk_aversion(market_prices['ticker'])
delta

9.802403162030824

In [39]:
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)
market_prior

LKOH     0.235540
NLMK     0.250621
AGRO     0.404671
MAGN     0.222163
TATN     0.175982
SIBN     0.177426
BANEP    0.306354
SNGSP    0.265854
POSI     0.254061
SBER     0.201803
CHMF     0.241332
ROSN     0.239506
dtype: float64

In [41]:
# You don't have to provide views on all the assets
viewdict = {
    "POSI": 0.10,
}

confidences = [
    0.6
]
bl = BlackLittermanModel(S, pi=market_prior, absolute_views=viewdict)

In [43]:
intervals = [
    (0.1, 0.4),
]

variances = []
for lb, ub in intervals:
    sigma = (ub - lb)/2
    variances.append(sigma ** 2)

print(variances)
omega = np.diag(variances)

[0.022500000000000006]


In [44]:
# We are using the shortcut to automatically compute market-implied prior
bl = BlackLittermanModel(S, pi="market", market_caps=mcaps, risk_aversion=delta,
                        absolute_views=viewdict, omega=omega)

In [45]:
# Posterior estimate of returns
ret_bl = bl.bl_returns()
ret_bl

LKOH     0.210907
NLMK     0.225580
AGRO     0.377385
MAGN     0.198196
TATN     0.153239
SIBN     0.155086
BANEP    0.281036
SNGSP    0.241803
POSI     0.213216
SBER     0.178689
CHMF     0.218008
ROSN     0.216051
dtype: float64

In [46]:
rets_df = pd.DataFrame([market_prior, ret_bl, pd.Series(viewdict)], 
             index=["Prior", "Posterior", "Views"]).T
rets_df

,Prior,Posterior,Views
LKOH,0.235540,0.210907,NaN
NLMK,0.250621,0.225580,NaN
AGRO,0.404671,0.377385,NaN
MAGN,0.222163,0.198196,NaN
TATN,0.175982,0.153239,NaN
SIBN,0.177426,0.155086,NaN
BANEP,0.306354,0.281036,NaN
SNGSP,0.265854,0.241803,NaN
POSI,0.254061,0.213216,0.1
SBER,0.201803,0.178689,NaN


In [49]:
S_bl = bl.bl_cov()


In [50]:
from pypfopt import EfficientFrontier, objective_functions

ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()
weights
     

D:\Soft\Anaconda\envs\t\Lib\site-packages\pypfopt\efficient_frontier\efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


OrderedDict([('LKOH', 0.07897),
             ('NLMK', 0.08567),
             ('AGRO', 0.14173),
             ('MAGN', 0.07393),
             ('TATN', 0.05579),
             ('SIBN', 0.0562),
             ('BANEP', 0.10686),
             ('SNGSP', 0.09078),
             ('POSI', 0.08075),
             ('SBER', 0.06611),
             ('CHMF', 0.08197),
             ('ROSN', 0.08124)])

In [52]:
from pypfopt import DiscreteAllocation

da = DiscreteAllocation(weights, dfp.iloc[-1], total_portfolio_value=100000)
alloc, leftover = da.lp_portfolio()
print(f"Leftover: ${leftover:.2f}")
alloc

Leftover: $703.29


D:\Soft\Anaconda\envs\t\Lib\site-packages\cvxpy\problems\problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


{'LKOH': 1,
 'NLMK': 39,
 'AGRO': 10,
 'MAGN': 130,
 'TATN': 8,
 'SIBN': 8,
 'BANEP': 5,
 'SNGSP': 141,
 'POSI': 3,
 'SBER': 22,
 'CHMF': 4,
 'ROSN': 15}

### Model

## Load all shares

In [7]:
candles = data_reader.candles[0][1]

In [10]:
tink.get_price(candles, "close")

,ticker
date,
2024-02-16,104.36
2024-02-19,105.54
2024-02-20,105.76
2024-02-21,102.60
2024-02-22,100.24
2024-02-26,102.78
2024-02-27,104.04
2024-02-28,103.80
2024-02-29,105.48


In [6]:
from dataload import ReadData

data_reader = ReadData(ETF)
data_reader.read_id_base()
data_reader.read_candles(days = 50, verbose = False)
#data_reader.to_df()
#data_reader.save('portfolio_prices.csv')

ABIO
ABRD
AFKS
AFLT
AGRO
AKRN
ALRS
AMEZ
APTK
AQUA
ASTR
BANE
BANEP
BELU
BLNG
BSPB
CARM
CBOM
CHMF
CHMK
CIAN
CNTL
CNTLP
DELI
DIAS
DSKY
DVEC
ELFV
ENPG
ETLN
EUTR
FEES
FESH
FIVE
FIXP
FLOT
GAZP
GCHE
GECO
GEMC
GLTR
GMKN
GRNT
GTRK
HHRU
HNFG
HYDR
IRAO
IRKT
KAZT
KAZTP
KLSB
KMAZ
KRKNP
KROT
KZIZ
KZIZP
KZOS
KZOSP
LEAS
LENT
LIFE
LKOH
LNZL
LNZLP
LSNG
LSNGP
LSRG
MAGN
MDMG
MGKL
MGNT
MGTSP
MOEX
MRKC
MRKP
MRKS
MRKU
MRKV
MRKY
MRKZ
MSNG
MSRS
MSTT
MTLR
MTLRP
MTSS
MVID
NKHP
NKNC
NKNCP
NLMK
NMTP
NOMP
NOMPP
NSVZ
NTZL
NVTK
OBNE
OBNEP
OGKB
OKEY
ORUP
OZON
PHOR
PIKK
PLZL
PMSB
PMSBP
POLY
POSI
PRFN
QIWI
RASP
RBCM
RENI
RKKE
RNFT
ROLO
ROSN
RTKM
RTKMP
RUAL
SBER
SBERP
SELG
SFIN
SFTL
SGZH
SIBN
SLAV
SMLT
SNGS
SNGSP
SOFL
SPBE
SVAV
SVCB
TATN
TATNP
TCSG
TGKA
TGKB
TGKBP
TGKJ
TGKN
TRMK
TRNFP
TTLK
UDMN
UFOSP
UGLD
UNAC
UNKL
UPRO
UWGN
VEON-RX
VKCO
VRSB
VSMO
VTBR
WUSH
YAKG
YNDX
ZILLP


In [16]:
prices_path = Path(TINK_DATA, 'portfolio_prices.csv')
with open(prices_path) as f:
    df_full = pd.read_csv(prices_path, index_col='date')
df_full

,ABIO,ABRD,AFKS,AFLT,AGRO,AKRN,ALRS,AMEZ,APTK,AQUA,...,UWGN,VEON-RX,VKCO,VRSB,VSMO,VTBR,WUSH,YAKG,YNDX,ZILLP
date,,,,,,,,,,,,,,,,,,,,,
2024-02-16,105.00,276.6,18.220,39.28,1420.0,18140.0,68.40,71.94,15.280,895.5,...,66.9,42.70,677.8,710.0,40100.0,0.024415,264.23,103.00,3445.0,2792.00
2024-02-19,104.40,273.8,18.150,38.61,1422.0,18038.0,68.78,71.00,15.050,884.0,...,63.0,42.95,660.0,699.0,39260.0,0.024360,263.18,103.60,3424.8,2712.01
2024-02-20,105.98,271.0,17.893,38.89,1400.0,18096.0,69.61,72.00,15.350,892.0,...,63.1,41.90,660.4,698.0,39340.0,0.024975,264.00,100.10,3311.0,2800.00
2024-02-21,105.00,261.0,17.500,37.74,1364.0,17984.0,69.81,69.32,15.030,870.0,...,60.3,40.55,644.0,660.5,38200.0,0.023600,255.00,97.45,3314.6,2615.03
2024-02-22,101.64,254.4,17.601,36.71,1346.0,17904.0,69.80,67.60,14.324,864.0,...,54.4,40.00,642.0,632.5,36840.0,0.023135,252.11,95.40,3255.0,2721.00
2024-02-26,101.96,261.0,18.000,37.60,1362.2,17962.0,70.89,69.86,14.332,880.0,...,57.5,43.00,653.0,656.0,37020.0,0.023850,258.01,95.40,3345.0,2819.99
2024-02-27,102.78,262.0,18.147,38.31,1418.0,18100.0,71.40,70.16,14.598,884.5,...,62.2,41.95,660.0,677.0,38800.0,0.023610,268.00,98.45,3415.0,2815.00
2024-02-28,104.04,266.2,18.029,37.72,1451.0,17952.0,72.85,69.72,14.574,879.5,...,62.9,42.55,651.2,680.5,37640.0,0.023400,266.90,97.65,3365.8,2754.00
2024-02-29,104.00,271.0,18.031,37.61,1423.4,17924.0,70.70,70.50,14.650,905.0,...,60.6,40.60,642.0,671.5,37320.0,0.023325,266.81,98.95,3384.0,2781.00


#### Drops Momentum

In [17]:
dfp = df_full.copy()
drops = ["GTRK", "NTZL", "LSRG", 'TGKBP', 'FIVE', "VEON-RX"]
columns = [x for x in dfp.columns if x not in drops]
dfp = dfp[columns]

#### Drops Base

In [17]:
dfp = df_full.copy()
index_assets = pd.read_csv('index_assets.csv')['asset'].values.tolist()
columns = [x for x in dfp.columns if x in index_assets]

drops = ['FIVE']
columns = [x for x in dfp.columns if x not in drops]

dfp = dfp[columns]

### RiskFolio

In [18]:
import riskfolio as rp

rms= ['MV', 'MSV',  'CVaR']

lookback = 30
df_period = dfp[-lookback:]

def riskfolio_weights(df_period, rm , obj):
    """
        obj - Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    """
    Y = df_period.pct_change().dropna()

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)
    port.solvers = ['MOSEK']
    # Calculating optimum portfolio

    # Select method and estimate input parameters:

    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Estimate optimal portfolio:

    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    # First we need to delete the cardinality constraint
    port.card = None 

    # Then we need to set the constraint on the minimum number of effective assets
    port.nea = 10
    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    w = w[w.weights > 0.01]
    return w

#print_data(dfp)
df_period = dfp.dropna(axis = 1)

dfw = riskfolio_weights(df_period, 'CVaR', 'MaxRet')
dfw

You must convert self.cov to a positive definite matrix


,weights
AFKS,0.054933944
AFLT,0.034769295
ASTR,0.025484625
CIAN,0.028878670
DELI,0.037141005
HHRU,0.044235481
LENT,0.049058222
OZON,0.030534352
PLZL,0.018282150
POSI,0.038139065


In [20]:
df_port['sums'] = df_port.quantity * df_port.price
sum_to_allocate = df_port.sums.sum() - 500
dfx = final_sums(dfw, sum_to_allocate, 50)
dfx['lot'] = 1
inds = dfx.index.values.tolist()

x = base_ru[base_ru['ticker'].isin (inds)]
s = x[['ticker', 'lot']].set_index('ticker')
dfx['lot'] = s

In [21]:
prices = dfp.iloc[-1].T.loc[dfx.index]
dfx["price"] = prices
dfx["lot_quantity"] = np.round(dfx.weights/ (dfx.price * dfx.lot)).astype(int)
dfx["quantity"] =  dfx.lot * dfx.lot_quantity
dfx["sum"]= dfx.price * dfx.lot_quantity * dfx.lot
dfx = dfx.sort_values("weights", ascending = False)
dfx.to_csv("t.csv")
dfx = dfx.reset_index()
dfx.columns = ['ticker', 'weights', 'lot', 'price', 'lot_quantity','quantity', 'sums']

dfx

,ticker,weights,lot,price,lot_quantity,quantity,sums
0,RBCM,22490.0,100,16.876,13,1300,21938.8
1,SFIN,19010.0,1,1852.000,10,10,18520.0
2,RNFT,11160.0,1,235.900,47,47,11087.3
3,AFKS,6030.0,100,23.805,3,300,7141.5
4,LENT,5380.0,1,998.500,5,5,4992.5
5,HHRU,4850.0,1,4573.000,1,1,4573.0
6,POSI,4180.0,1,2799.600,1,1,2799.6
7,UGLD,4110.0,1000,0.940,4,4000,3760.0
8,DELI,4070.0,1,349.000,12,12,4188.0
9,AFLT,3810.0,10,48.430,8,80,3874.4


In [22]:
df_port

,figi,ticker,name,quantity,lot_quantity,price,sums
4,RUB000UTSTOM,0-RUB,None,47946,47946,1.00,47946.0
6,TCS10A0JNAB6,ABIO,Артген,10,1,118.86,1188.6
1,BBG004S68614,AFKS,АФК Система,200,2,24.00,4800.0
0,BBG004S683W7,AFLT,Аэрофлот,80,8,49.27,3941.6
3,BBG012YQ6P43,CIAN,Циан АДР,3,3,859.00,2577.0
9,TCS00A107J11,DELI,Делимобиль,8,8,346.00,2768.0
7,TCS2207L1061,HHRU,HeadHunter Group PLC,1,1,4430.00,4430.0
2,BBG0063FKTD9,LENT,Лента,7,7,962.50,6737.5
8,TCS00A103X66,POSI,Positive Technologies,1,1,2817.60,2817.6
5,BBG00F9XX7H4,RNFT,РуссНефть,135,135,244.30,32980.5


In [23]:
def df_to_dict(dfx):
    d = {row.ticker:row.lot_quantity for _,row in dfx.iterrows()}
    sd = dict(sorted(d.items()))
    return  sd

old_port = df_to_dict(df_port)
new_port = df_to_dict(dfx)
old_port

{'0-RUB': 47946,
 'ABIO': 1,
 'AFKS': 2,
 'AFLT': 8,
 'CIAN': 3,
 'DELI': 8,
 'HHRU': 1,
 'LENT': 7,
 'POSI': 1,
 'RNFT': 135}

In [24]:
new_port

{'AFKS': 3,
 'AFLT': 8,
 'ASTR': 4,
 'CIAN': 4,
 'DELI': 12,
 'HHRU': 1,
 'LENT': 5,
 'OZON': 1,
 'PLZL': 0,
 'POSI': 1,
 'RBCM': 13,
 'RNFT': 47,
 'RTKM': 2,
 'RTKMP': 2,
 'SELG': 2,
 'SFIN': 10,
 'UGLD': 4,
 'WUSH': 10,
 'YNDX': 0}

In [45]:


prd = dict(data_reader.candles)
for pos in new_port:
    prd[pos]
    last_close_price = prd[pos][-1][4]
    lot = base_ru[base_ru.ticker == pos].lot.values[0]
    lot_qty = new_port[pos]
    qty = lot * lot_qty
    sums = last_close_price * qty
    print(pos, lot_qty, qty, last_close_price, sums)

AFKS 3 300 23.986 7195.8
AFLT 8 80 49.36 3948.8
ASTR 4 4 663.5 2654.0
CIAN 4 4 862.4 3449.6
DELI 12 12 342.2 4106.4
HHRU 1 1 4430.0 4430.0
LENT 5 5 962.5 4812.5
OZON 1 1 3866.5 3866.5
PLZL 0 0 13363.5 0.0
POSI 1 1 2812.8 2812.8
RBCM 13 1300 15.6 20280.0
RNFT 47 47 243.8 11458.6
RTKM 2 20 97.9 1958.0
RTKMP 2 20 85.65 1713.0
SELG 2 20 79.97 1599.4
SFIN 10 10 1788.4 17884.0
UGLD 4 4000 0.9417 3766.7999999999997
WUSH 10 10 314.7 3147.0
YNDX 0 0 3995.0 0.0


In [37]:
def df_to_dict(df):
  """
  Преобразовать Pandas DataFrame в словарь.

  Args:
      df: Pandas DataFrame.

  Returns:
      Словарь, где ключи - значения из первой колонки, 
      а значения - значения из второй колонки.
  """

  if len(df.columns) != 2:
    raise ValueError("DataFrame должен иметь 2 колонки")

    return dict(zip(df.iloc[:, 0], df.iloc[:, 1]))

base_ru[['ticker', 'lot']]

,ticker,lot
26,KZIZP,1
52,IRKT,100
68,VSMO,1
73,UNAC,1000
91,VKCO,1
...,...,...
1926,NMTP,100
1930,TGKA,100000
1947,CNTLP,100
1960,MTLRP,10


In [28]:
def calculate_portfolio_difference(old_portfolio, new_portfolio):
    """
    Рассчитать разницу между двумя портфелями.

    Args:
        old_portfolio: Словарь, где ключом является тикер, а значением - количество акций.
        new_portfolio: Словарь, где ключом является тикер, а значением - количество акций.

    Returns:
        Словарь, где ключом является тикер, а значением - разница между количеством акций в 
        новых и старых портфелях.
    """

    difference = {}
    for ticker in new_portfolio:
        if ticker in old_portfolio:
            difference[ticker] = new_portfolio[ticker] - old_portfolio[ticker]
        else:
            difference[ticker] = new_portfolio[ticker]

    for ticker in old_portfolio:
        if ticker not in new_portfolio:
            difference[ticker] = -old_portfolio[ticker]
    # Сортировка по значению, по возрастанию
    sorted_diff = sorted(difference.items(), key=lambda x: x[1])

    return sorted_diff

In [29]:
rebalance = calculate_portfolio_difference(old_port, new_port)

flag_rebalance = True

In [31]:
for asset, qty in rebalance:
    if asset is None:
        continue
        
    print(asset, qty)

0-RUB -47946
RNFT -88
LENT -2
ABIO -1
AFLT 0
HHRU 0
PLZL 0
POSI 0
YNDX 0
AFKS 1
CIAN 1
OZON 1
RTKM 2
RTKMP 2
SELG 2
ASTR 4
DELI 4
UGLD 4
SFIN 10
WUSH 10
RBCM 13


In [33]:
s = slice(1,2)
x = [1,2,3,4,5]
x[s]

[2]

In [58]:
from tinkoff.invest import OrderDirection, OrderType
residuals = []
if flag_rebalance:
    with Client(token) as client:

        for asset, qty in rebalance[12:]:
            print(asset)
            if asset is None:
                continue
                
            figi = tink.ticker_to_figi(asset, base)
            trading_status = client.market_data.get_trading_status(
                figi=figi
            )
            
            if trading_status.market_order_available_flag and trading_status.api_trade_available_flag:
                if qty < 0:
                    resp = client.orders.post_order(figi=figi,
                                quantity= -qty,
                                direction=OrderDirection.ORDER_DIRECTION_SELL,
                                account_id=account_id,
                                order_type=OrderType.ORDER_TYPE_MARKET,)
                elif qty > 0:
                    resp = client.orders.post_order(figi=figi,
                        quantity=qty,
                        direction=OrderDirection.ORDER_DIRECTION_BUY,
                        account_id=account_id,
                        order_type=OrderType.ORDER_TYPE_MARKET,)
            else:
                print("Не доступно")
                residuals.append((asset, qty))
    flag_rebalance = False

LENT
Не доступно
SELG
CIAN
Не доступно
GLTR
ASTR
AFLT
UGLD
Не доступно
DELI
Не доступно
RBCM
Не доступно
SFIN
Не доступно
RNFT


In [59]:
residuals

[('LENT', 2),
 ('CIAN', 3),
 ('UGLD', 6),
 ('DELI', 10),
 ('RBCM', 12),
 ('SFIN', 13)]